## Results Section 6.6: Relative Phase Operators
We present the results for Section 6.6. Specifically, we reproduce figure 6 here.

### Imports and setup

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from analyzer import main_analysis
import os
from matplotlib import rc
from qiskit import QuantumCircuit, QuantumRegister

In [ ]:
source_file = "../../data/times.csv"
source_file_optimization = "../../data/optimization.csv"
source_output = '../../data/output'
output_folder = "../../data/paper"

### Helper Functions

In [ ]:
def ccx_tdepth():
    circ = QuantumCircuit(3)
    circ.h(2)
    circ.t(0)
    circ.t(1)
    circ.t(2)
    circ.cx(1, 0)
    circ.cx(2, 1)
    circ.cx(0, 2)
    circ.tdg(1)
    circ.cx(0, 1)
    circ.tdg(0)
    circ.tdg(1)
    circ.t(2)
    circ.cx(2, 1)
    circ.cx(0, 2)
    circ.h(2)
    circ.cx(1, 0)
    return circ.to_gate()

def ccx_tcount():
    circ = QuantumCircuit(3)
    circ.h(2)
    circ.cx(1, 2)
    circ.tdg(2)
    circ.cx(0, 2)
    circ.t(2)
    circ.cx(1, 2)
    circ.t(1)
    circ.tdg(2)
    circ.cx(0, 2)
    circ.cx(0, 1)
    circ.t(2)
    circ.h(2)
    circ.t(0)
    circ.tdg(1)
    circ.cx(0, 1)
    return circ.to_gate()

def cirq_carry_gate(ccxg):
    # https://github.com/quantumlib/Cirq/blob/main/examples/basic_arithmetic.py
    carry_circ = QuantumCircuit(4)
    carry_circ.append(ccxg, [1, 2, 3])
    carry_circ.cx(1, 2)
    carry_circ.append(ccxg, [0, 2, 3])
    return carry_circ.to_gate()

def cirq_uncarry_gate(ccxg):
    # https://github.com/quantumlib/Cirq/blob/main/examples/basic_arithmetic.py
    carry_circ = QuantumCircuit(4)
    carry_circ.append(ccxg, [0, 2, 3])
    carry_circ.cx(1, 2)
    carry_circ.append(ccxg, [1, 2, 3])
    return carry_circ.to_gate()

def build_adder(n, carry_gate, uncarry_gate):
    # https://github.com/quantumlib/Cirq/blob/main/examples/basic_arithmetic.py
    def sum_gate():
        sum_circ = QuantumCircuit(3)
        (c0, a, b) = (sum_circ.qubits[0], sum_circ.qubits[1], sum_circ.qubits[2])
        sum_circ.cx(a, b)
        sum_circ.cx(c0, b)
        return sum_circ.to_gate()

    a = QuantumRegister(n, name = "a")
    b = QuantumRegister(n, name = "b")
    c = QuantumRegister(n, name = "c")

    circuit = QuantumCircuit(a, b, c)

    for i in range(n-1):
        circuit.append(carry_gate, [c[i], a[i], b[i], c[i+1]])
    circuit.append(sum_gate(), [c[n-1], a[n-1], b[n-1]])
    for i in range(n-2, -1, -1):
        circuit.append(uncarry_gate, [c[i], a[i], b[i], c[i+1]])
        circuit.append(sum_gate(), [c[i], a[i], b[i]])

    return circuit

def plot_fig6(size, nb_gates_cirq, nb_gates_ours, label):
    fig, ax = plt.subplots(figsize=(3,3))
    current_palette = sns.color_palette("Dark2", 2)
    sns.despine(fig, left=True, bottom=True)
    ax.set_facecolor((0.95, 0.95, 0.95))
    ax.plot(size, nb_gates_cirq, color=current_palette[1])
    ax.plot(size, nb_gates_ours, color=current_palette[0])
    ax.title.set(x=0.05)
    ax.set_title(label)
    ax.set_xlabel('# Qubits', fontsize=12)
    fig.tight_layout()
    os.makedirs(os.path.join(output_folder, 'fig6'), exist_ok=True)
    fig.savefig(os.path.join(output_folder, 'fig6', f'{label}.pdf'))

In [ ]:
_, _, _, _, optimal_carry_t, _ = main_analysis(os.path.join(source_output, '66', 'carry_relative'))
_, _, _, _, _, optimal_carry_cx = main_analysis(os.path.join(source_output, '66', 'carry_relative_cnot'))
optimal_uncarry_t = optimal_carry_t.circuit.copy().inverse().to_gate()
optimal_carry_t = optimal_carry_t.circuit.to_gate()
optimal_uncarry_cx = optimal_carry_cx.circuit.copy().inverse().to_gate()
optimal_carry_cx = optimal_carry_cx.circuit.to_gate()

x = np.arange(2, 21, 1)
y_tdepth = []
for nq in x:
    adder_circ = build_adder(nq, cirq_carry_gate(ccx_tdepth()), cirq_uncarry_gate(ccx_tdepth()))
    decomp_circ = adder_circ.decompose().decompose()
    #print(decomp_circ)
    depth = decomp_circ.depth(lambda g: g[0].name in ['t', 'tdg'])
    y_tdepth.append(depth)

y_tcount = []
for nq in x:
    adder_circ = build_adder(nq, cirq_carry_gate(ccx_tcount()), cirq_uncarry_gate(ccx_tcount()))
    decomp_circ = adder_circ.decompose().decompose()
    #print(decomp_circ)
    count_ops = decomp_circ.count_ops()
    count = count_ops['t'] + count_ops['tdg']
    y_tcount.append(count)

y_cx_depth = []
for nq in x:
    adder_circ = build_adder(nq, cirq_carry_gate(ccx_tcount()), cirq_uncarry_gate(ccx_tcount()))
    decomp_circ = adder_circ.decompose().decompose()
    #print(decomp_circ)
    depth = decomp_circ.depth(lambda g: g[0].name in ['cx'])
    y_cx_depth.append(depth)

y_cx_count = []
for nq in x:
    adder_circ = build_adder(nq, cirq_carry_gate(ccx_tcount()), cirq_uncarry_gate(ccx_tcount()))
    decomp_circ = adder_circ.decompose().decompose()
    #print(decomp_circ)
    count_ops = decomp_circ.count_ops()
    count = count_ops['cx']
    y_cx_count.append(count)

y_tdepth_optimal = []
y_tcount_optimal = []
for nq in x:
    adder_circ = build_adder(nq, optimal_carry_t, optimal_uncarry_t)
    decomp_circ = adder_circ.decompose()
    #print(decomp_circ)
    depth = decomp_circ.depth(lambda g: g[0].name in ['t', 'tdg'])
    count_ops = decomp_circ.count_ops()
    count = count_ops['t'] + count_ops['tdg']
    y_tdepth_optimal.append(depth)
    y_tcount_optimal.append(count)

y_cx_depth_optimal = []
y_cx_count_optimal = []

for nq in x:
    adder_circ = build_adder(nq, optimal_carry_cx, optimal_uncarry_cx)
    decomp_circ = adder_circ.decompose()
    #print(decomp_circ)
    depth = decomp_circ.depth(lambda g: g[0].name in ['cx'])
    count_ops = decomp_circ.count_ops()
    count = count_ops['cx']
    y_cx_depth_optimal.append(depth)
    y_cx_count_optimal.append(count)

plot_fig6(x, y_tdepth, y_tdepth_optimal, 'T-Depth')
plot_fig6(x, y_tcount, y_tcount_optimal, 'T-Count')
plot_fig6(x, y_cx_depth, y_cx_depth_optimal, 'CX-Depth')
plot_fig6(x, y_cx_count, y_cx_count_optimal, 'CX-Count')